In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/multi/0422

/content/drive/MyDrive/multi/0422


In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
up = files.upload()

### 피마 인디언 당뇨병 예측

In [4]:
df = pd.read_csv('./data/pima-indians-diabetes.csv', skiprows=9, header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df2 = pd.read_csv('./data/diabetes.csv')
df2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
df.columns = ['P', 'G', 'BP', 'ST', 'IS', 'BMI', 'DPF', 'Age', 'Class']
df.head()

,P,G,BP,ST,IS,BMI,DPF,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
# 결측값 확인
df.isnull().sum().any()

False

In [30]:
X = df.iloc[:, :-1].values
y = df.iloc[:,-1].values

In [31]:
print(X.shape, y.shape)

(768, 8) (768,)


- Train/Test data 분리

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2022
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [35]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([400, 214]))

In [36]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([100,  54]))

- Decision Tree Classifier

In [38]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=2022)
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [39]:
params = {
    'max_depth':[2,3,4,5,6],
    'min_samples_split':[2,3,4]
}

- GridSearchCV

In [42]:
from sklearn.model_selection import GridSearchCV

grid_dt = GridSearchCV(
    dtc, params, scoring='accuracy', cv=5
)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2022),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [43]:
grid_dt.best_params_

{'max_depth': 5, 'min_samples_split': 3}

In [44]:
dtc_best = grid_dt.best_estimator_
dtc_best.score(X_test, y_test)

0.6558441558441559

- 실제 적용

In [48]:
X_test

array([[  0.   , 135.   ,  94.   , ...,  40.6  ,   0.284,  26.   ],
       [  4.   , 183.   ,   0.   , ...,  28.4  ,   0.212,  36.   ],
       [  2.   ,  91.   ,  62.   , ...,  27.3  ,   0.525,  22.   ],
       ...,
       [  0.   , 107.   ,  62.   , ...,  36.6  ,   0.757,  25.   ],
       [  1.   , 116.   ,  78.   , ...,  36.1  ,   0.496,  25.   ],
       [  1.   , 136.   ,  74.   , ...,  37.4  ,   0.399,  24.   ]])

In [51]:
X_test.shape

(154, 8)

In [54]:
# shape을 맞춰주기
pred = dtc_best.predict(X_test[0].reshape(1,-1))
print('양성' if pred[0] == 1 else '음성')

음성
